## Setup

In [33]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# load model architecture
from tep.modelUtils import load_architecture
model = load_architecture('models/tl/baseline.json')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
seq_input (InputLayer)       (None, 32)                0         
_________________________________________________________________
word_emb (Embedding)         (None, 32, 100)           43224100  
_________________________________________________________________
input_dropout (Dropout)      (None, 32, 100)           0         
_________________________________________________________________
padding_1 (ZeroPadding1D)    (None, 34, 100)           0         
_________________________________________________________________
conv_1 (Conv1D)              (None, 32, 128)           38528     
_________________________________________________________________
pool_1 (MaxPooling1D)        (None, 16, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 128)           0         
__________

In [2]:
# set model weights
model.load_weights('models/tl/baseline.hdf5')

In [4]:
# load data and labels
from tep.utils import load_array
seqs = load_array('data/sentiment_data/seq_32.bc')
labels = load_array('data/sentiment_data/labels.bc')
print(seqs.shape)
print(labels.shape)

(1600000, 32)
(1600000,)


In [8]:
# load word index
import json
word_index_file = open('data/sentiment_data/word_index.json')
word_index_str = word_index_file.read()
word_index = json.loads(word_index_str)
type(word_index)

dict

In [9]:
# compile the model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])

In [11]:
# test the model
from tep.trainUtils import one_hot_encoding
oh_labels = one_hot_encoding(labels, 2)
model.evaluate(x=seqs[:10000], y=oh_labels[:10000], batch_size=256)

10000/10000 [==============================] - 2s 212us/step


[0.34918261456489563, 0.84550000000000003]

## Get hidden layer activations

In [93]:
from tep.featureVisualization import FeatureVisualizer
fv = FeatureVisualizer(model, word_index)

In [94]:
activations = fv.get_activations('conv_1', seqs[:1])

In [98]:
model.predict(seqs[:1])

array([[ 0.21656498,  0.78343499]], dtype=float32)

In [102]:
fv.analyze_example(seqs[0], 'conv_1', num_results=10)

[{'activation': 2.4955232,
  'kernel': 58,
  'position': 28,
  'words': "me,it's my birthday"},
 {'activation': 1.7044402,
  'kernel': 64,
  'position': 23,
  'words': '<user> aww,wish you'},
 {'activation': 1.6564744,
  'kernel': 43,
  'position': 30,
  'words': 'birthday on monday'},
 {'activation': 1.6178091,
  'kernel': 127,
  'position': 26,
  'words': "could greet me,it's"},
 {'activation': 1.5845333,
  'kernel': 122,
  'position': 28,
  'words': "me,it's my birthday"},
 {'activation': 1.5660692,
  'kernel': 15,
  'position': 29,
  'words': 'my birthday on'},
 {'activation': 1.5241085,
  'kernel': 120,
  'position': 29,
  'words': 'my birthday on'},
 {'activation': 1.5133868,
  'kernel': 33,
  'position': 29,
  'words': 'my birthday on'},
 {'activation': 1.4497467,
  'kernel': 72,
  'position': 24,
  'words': 'aww,wish you could'},
 {'activation': 1.4492041,
  'kernel': 100,
  'position': 24,
  'words': 'aww,wish you could'}]

In [45]:
type(activations)

numpy.ndarray

In [46]:
activations.shape

(1, 32, 128)

In [47]:
seq = seqs[0]

In [48]:
seq

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      2, 150999,
           12,    148,   4962, 101456,     10,    286,     20,    363], dtype=int32)

In [54]:
fv.get_words(seq, 30, 3)

'birthday on monday'

In [55]:
import numpy as np

In [56]:
activations = np.squeeze(activations)
activations.shape

(32, 128)

In [79]:
xs, ys = np.unravel_index(np.argsort(activations, axis=None), activations.shape)
print(xs)
print(ys)

[ 0 19 19 ..., 30 23 28]
[  0 123 124 ...,  43  64  58]


In [90]:
a = []
for x, y in zip(xs, ys):
    a.append((x,y))
len(a)

4096

In [92]:
a = np.flip(a, axis=0)
print(activations[a[0]])
print(activations.max())

IndexError: index 58 is out of bounds for axis 0 with size 32

In [84]:
sorted_idx = np.flip(list(zip(xs, ys)), axis=0)
sorted_idx

array([[ 28,  58],
       [ 23,  64],
       [ 30,  43],
       ..., 
       [ 19, 124],
       [ 19, 123],
       [  0,   0]])

In [85]:
print(activations[sorted_idx[0]])
print(activations.max())

IndexError: index 58 is out of bounds for axis 0 with size 32

In [78]:
np.flip(sorted_x, axis=0)

array([28, 23, 30, ..., 19, 19,  0])

In [70]:
print(activations[28, 58])
print(activations[23, 64])
print(activations.max())

2.49552
1.70444
2.49552
